In [1]:
!pip install transformers datasets torch peft accelerate bitsandbytes tensorboard pandas matplotlib seaborn nltk rouge

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import os
import random
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    set_seed
)
from peft import (
    get_peft_model,
    LoraConfig,
    TaskType,
    PeftModel
)
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

#Set seed for resproducibility
set_seed(42)
#-------------------------------------------
#1. Create a custom Sentiment Dataset
#-------------------------------------------

def generate_sentiment_dataset(num_samples=100):
  """Generate a simple dataset with text and sentiment labels."""

  #Templates for positive and negative sentences
  positive_templates = [
        "I absolutely loved {item}. It was {adjective}!",
        "The {item} exceeded my expectations, truly {adjective}.",
        "What a wonderful {item}! I'm so {feeling} about it.",
        "{item} was fantastic! I would definitely recommend it to anyone.",
        "I'm very impressed with {item}. It's {adjective} and worth every penny.",
        "The {item} made my day. It's simply {adjective}.",
        "I had a great experience with {item}. It's {adjective}!",
        "The {item} was a delight. I'm feeling {feeling} after using it.",
        "I can't praise {item} enough! It's {adjective} in every way.",
        "The {item} brings so much joy. I'm {feeling} about my purchase."
    ]

  negative_templates = [
        "I was disappointed with {item}. It was {adjective}.",
        "The {item} fell short of my expectations, truly {adjective}.",
        "What a terrible {item}! I'm so {feeling} about it.",
        "{item} was awful! I would definitely not recommend it to anyone.",
        "I'm very unimpressed with {item}. It's {adjective} and a waste of money.",
        "The {item} ruined my day. It's simply {adjective}.",
        "I had a poor experience with {item}. It's {adjective}!",
        "The {item} was a nightmare. I'm feeling {feeling} after using it.",
        "I can't criticize {item} enough! It's {adjective} in every way.",
        "The {item} brings so much frustration. I'm {feeling} about my purchase."
    ]

  items = [
        "product", "service", "movie", "book", "restaurant", "hotel",
        "experience", "app", "device", "food", "coffee", "concert",
        "vacation", "phone", "laptop", "customer support", "delivery",
        "interface", "game", "website"
    ]

  positive_adjectives = [
        "amazing", "fantastic", "excellent", "outstanding", "perfect",
        "brilliant", "incredible", "superb", "wonderful", "exceptional"
    ]

  negative_adjectives = [
        "disappointing", "terrible", "awful", "poor", "subpar",
        "horrible", "dreadful", "mediocre", "unacceptable", "frustrating"
    ]

  positive_feelings = [
        "happy", "delighted", "thrilled", "excited", "pleased",
        "satisfied", "impressed", "grateful", "ecstatic", "contented"
    ]

  negative_feelings = [
        "upset", "frustrated", "annoyed", "disappointed", "angry",
        "displeased", "irritated", "dissatisfied", "unhappy", "regretful"
    ]

# generate balanced dataset
  texts = []
  labels = []

  for _ in range(num_samples // 2):
    #Generate positive example
    template = random.choice(positive_templates)
    item = random.choice(items)
    adjective = random.choice(positive_adjectives)
    feeling = random.choice(positive_feelings)
    text = template.format(item=item, adjective=adjective, feeling=feeling)
    texts.append(text)
    labels.append("positive")

    # Generate negative example
    template = random.choice(negative_templates)
    item = random.choice(items)
    adjective = random.choice(negative_adjectives)
    feeling = random.choice(negative_feelings)
    text = template.format(item=item, adjective=adjective, feeling=feeling)
    texts.append(text)
    labels.append("negative")


  # Convert to DataFrame
  df = pd.DataFrame({"text": texts, "sentiment": labels})

  #Shuffle the dataset
  df = df.sample(frac=1).reset_index(drop=True)

  return df

#Generate our sentiment dataset
sentiment_df = generate_sentiment_dataset(100)
print(f"Generated dataset with {len(sentiment_df)} examples")
print(sentiment_df.head())

#Split into train and test sets (80/20 split)
train_df = sentiment_df.sample(frac=0.8, random_state=42)
test_df = sentiment_df.drop(train_df.index)

print(f"Train set: {len(train_df)} examples")
print(f"Test set: {len(test_df)} examples")





Generated dataset with 100 examples
                                                text sentiment
0  The app fell short of my expectations, truly h...  negative
1  I can't criticize restaurant enough! It's frus...  negative
2    I absolutely loved service. It was exceptional!  positive
3  I can't criticize experience enough! It's subp...  negative
4  I can't praise website enough! It's incredible...  positive
Train set: 80 examples
Test set: 20 examples


In [5]:
#---------------------------------------------
#2. Prepare dataset for fine tuning
#---------------------------------------------

#Convert to Hugging face datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

#Configuration
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"# Using a smaller model for demonstration
OUTPUT_DIR ="./models/sentiment_lora_finetuned"
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs("./data",exist_ok = True)

#Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

# Format the dataset for instruction tuning
def format_instruction(example):
  """Format the example into an instruction tuninf format."""
  text = example["text"]
  sentiment = example["sentiment"]

  #Create instruction format
  instruction = f"###Instruction:\nClassify the sentiment of the following text as either 'positive' or 'negative'.\n\n### Text:\n{text}\n\n### Sentiment:\n{sentiment}"

  return {"formatted_text": instruction}

print("Formatting the dataset...")
#Apply formatting to the dataset
train_formatted = train_dataset.map(format_instruction)
test_formatted = test_dataset.map(format_instruction)

#Tokenize the dataset
def tokenize_function(examples):
  """Tokenize examples and handle truncation."""
  return tokenizer(
      examples["formatted_text"],
      truncation=True,
      max_length=512,
      padding="max_length",
  )

print("Tokenizing dataset...")
train_tokenized = train_formatted.map(
    tokenize_function,
    batched=True,
    remove_columns=train_formatted.column_names

    )

test_tokenized = test_formatted.map(
    tokenize_function,
    batched=True,
    remove_columns=test_formatted.column_names
)

#Save processed datasets
train_tokenized.save_to_disk("./data/sentiment_train")
test_tokenized.save_to_disk("./data/sentiment_test")


Formatting the dataset...


Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Tokenizing dataset...


Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/80 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/20 [00:00<?, ? examples/s]

In [6]:
#---------------------------------
#3. LoRA fine tuning
#---------------------------------

#Data collator for language modelling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
    )

#load model in full precision
print(f"Loading model: {MODEL_NAME} in full precision")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16, # Using float16 for efficiency but not quantization
    device_map="auto",
)

#Configure LoRA
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj","k_proj","v_proj","o_proj"],# Target attention modules
)

# Apply LoRA adapters to model
model = get_peft_model(model, peft_config)

# print trainable parameters info
def print_trainable_parameters(model):
  """Print the number of trainable parameters."""
  trainable_params = 0
  all_params = 0
  for _, param in model.named_parameters():
    all_params += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()

  print(
      f"TRainable params: {trainable_params} ({100 * trainable_params / all_params:.2f}% of all params)"
  )

print_trainable_parameters(model)

#Configure training arguments
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=True,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_dir=f"{OUTPUT_DIR}/logs",
    logging_steps=10,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    save_total_limit=1,
    fp16=True, # Still using mixed precision for efficiency
    report_to="tensorboard",
    push_to_hub=False,
)

#INitialise trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

print("Starting LoRA fine tuning...")
#Train the model
trainer.train()

print("LoRA fine-tuning complete!")

#save the final model
trainer.save_model(f"{OUTPUT_DIR}/final")
print(f"Model saved to {OUTPUT_DIR}/final")


Loading model: TinyLlama/TinyLlama-1.1B-Chat-v1.0 in full precision


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-6-c6977153728a>:68: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


TRainable params: 2252800 (0.20% of all params)
Starting LoRA fine tuning...


Epoch,Training Loss,Validation Loss
1,No log,1.793576
2,1.893500,1.419513
3,1.893500,1.274948


LoRA fine-tuning complete!
Model saved to ./models/sentiment_lora_finetuned/final
